# **FUNCTIONS**

Este notebook contiene las funciones principales (las 5 solicitadas en el caso) y secundarias que sirven como soporte para las principales como algoritmo representativo para la creación de los datasets finales guardados en 
formato CSV para su posterior lectura en el módulo principal (main.py) y sea usado para brindar el servicio al 
mundo entero.

Por lo tanto, se debe realizar la siguiente aclaración, si bien las secciones tienen el nombre, incluso la sintaxis
solicitada en el caso, no son los definitivos usados en el API framework, sino fueron puestos para servir de guía 
al momento de generar los CSV finales guardados en la carpeta "Data4_FuncReturn2API" y que serán cargados por el 
módulo principal main.py.

In [15]:
## Se importan las librerías necesarias para esta etapa de la definición de funciones y creación de CSV finales
import pandas as pd
import html
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
## Carga de archivos CSV generados en la etapa ETL-EDA

df_games = pd.read_csv("Data3_EDA_out\\EDA_games.csv")
df_reviews = pd.read_csv("Data3_EDA_out\\EDA_reviews.csv")
df_items = pd.read_csv("Data3_EDA_out\\EDA_items.csv")
df_Gral_UserRec = pd.read_csv("Data3_EDA_out\\SentimAnalysis.csv")


# PlayTimeGenre

def PlayTimeGenre( genero : str ): Debe devolver año con más horas jugadas para dicho género.


In [17]:
## Antes de crear la función, primero preparar el dataframe para mejorar el rendimiento de la función
playtime_item = df_items.groupby('item_id')['playtime_forever'].sum().reset_index()
df_merge = pd.merge(df_games, playtime_item, on='item_id', how='inner')
df_PlayTimeGenre = df_merge[["genres","release_year","playtime_forever"]]
df_PlayTimeGenre =df_PlayTimeGenre.dropna()

In [18]:
## CREACIÓN DE LA FUNCIÓN 1 PARA LA API
def PlayTimeGenre(genre : str):
    # Filtrar el DataFrame para obtener sÓlo las filas que contienen el género específico
    df_filtered  = df_PlayTimeGenre[df_PlayTimeGenre.genres.str.contains(genre)]
    df_grouped = df_filtered.groupby('release_year')['playtime_forever'].sum().reset_index()
    
    # Encontrar el año con la mayor suma de 'playtime_forever'
    df_grouped['release_year'] = df_grouped['release_year'].astype(int)
    sum_max_year = df_grouped.loc[df_grouped['playtime_forever'].idxmax(), 'release_year']
    
    return sum_max_year 

In [19]:
## CREACIÓN DEL CSV DE CONSULTA 1 OPTIMIZADA PARA LA API
df_PlayTimeForever=pd.read_csv('Data3_EDA_out\genres.csv')
df_PlayTimeForever['maxplaytime_year'] = df_PlayTimeForever['genre'].apply(lambda x: PlayTimeGenre(x))
df_PlayTimeForever.to_csv('Data4_FuncReturn2API\PlayTimeForever.csv',index=False)


# UserForGenre

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

In [20]:
## Antes de crear la función, primero preparar el dataframe para mejorar el rendimiento de la función
df_UserForGenre = pd.merge(df_items,df_games[["item_id","genres","release_year"]], on = "item_id")
df_UserForGenre =df_UserForGenre[["user_id","genres","release_year","playtime_forever"]]

# Eliminar las filas que no tienen minutos jugados para agilizar los procesos de iteración
df_UserForGenre=df_UserForGenre[df_UserForGenre.playtime_forever>0]

# Reemplazar los caracteres de escape presentes en la columna GENRES
df_UserForGenre.genres=df_UserForGenre.genres.apply(lambda x: html.unescape(x))

In [21]:
def UserForGenre(genre: str,retval=1):
    # Filtrar el DataFrame para el género especificado
    #filtro_genero = df_UserForGenre['genres'].str.contains(genre, case=False, na=False)
    df_filtered  = df_UserForGenre[df_UserForGenre.genres.str.contains(genre)]

    # Agrupar por 'user_id' y 'year', sumar las horas jugadas
    df_grouped = df_filtered.groupby(['user_id', 'release_year'])['playtime_forever'].sum().reset_index()
    
    # Conversión de las sumas de los minutos jugados a HORAS enteras redondeado
    df_grouped.playtime_forever=df_grouped.playtime_forever.apply(lambda x: round(x / 60))
    
    # Eliminar las filas que no tienen HORAS jugadas para agilizar los procesos de iteración
    df_grouped=df_grouped[df_grouped.playtime_forever>0]

    # Encontrar el usuario con la máxima suma de horas jugadas
    max_playtime_usr = df_grouped.loc[df_grouped.playtime_forever.idxmax(), 'user_id']
    if retval==1:
        return max_playtime_usr
    else:
    
        # Filtrar el DataFrame para el usuario con máxima suma de horas jugadas
        df_usuario = df_grouped[df_grouped.user_id == max_playtime_usr]

        # Crear el formato "Horas jugadas"
        resultado_final = [{'Año': int(row['release_year']), 'Horas': int(row['playtime_forever'])} for _, row in df_usuario.iterrows()]
    
        return f"Horas jugadas: {resultado_final}"


In [22]:
## CREACIÓN DEL CSV DE CONSULTA 2 OPTIMIZADA PARA LA API
df_UserForGenre_csv=pd.read_csv('Data3_EDA_out\genres.csv')
df_UserForGenre_csv['user'] = df_UserForGenre_csv['genre'].apply(lambda x: UserForGenre(x))
df_UserForGenre_csv['list_played_hours'] = df_UserForGenre_csv['genre'].apply(lambda x: UserForGenre(x,2))
df_UserForGenre_csv.to_csv('Data4_FuncReturn2API\\UsersForGenre.csv',index=False)

# UsersRecommend

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)


In [23]:
## Antes de crear la función, primero preparar el dataframe para mejorar el rendimiento de la función
df_UserRecommend = df_Gral_UserRec[df_Gral_UserRec.recommend]
df_UserRecommend = df_UserRecommend.groupby(['posted_year', 'app_name']).sum().reset_index()
df_UserRecommend.drop(columns=['recommend','item_id','user_id'], inplace=True)
df_UserRecommend['points'] = df_UserRecommend.Positivo+df_UserRecommend.Neutro
df_UserRecommend = df_UserRecommend.sort_values(by=['posted_year', 'points'],ascending=[True, False])

In [24]:
## CREACIÓN DE LA FUNCIÓN QUE ME DEVUELVE LOS JUEGOS TOP3, TANTO SUPERIORES COMO INFERIORES SEGÚN df_qry
def top3_games(year: int, df_qry, topnum=1):
    # filtrar los 3 juegos más recomendados del año ingresado
    games_top3=df_qry[(df_qry.posted_year == year)].head(3)

    # los 3 nombres de los juegos se envían a una lista
    lst_gametop3 =games_top3.app_name.to_list()

    if len(lst_gametop3)<topnum:
        # se devolverá vacío si no se han realizado más buenas o malas recomendaciones según el dataframe ingresado
        return '-'
    else:
        # se devolverá el número de lugar en el top según se indique con topnum
        return lst_gametop3[topnum-1]

In [25]:
## CREACIÓN DEL CSV DE CONSULTA 3 OPTIMIZADA PARA LA API
df_UR2API = df_UserRecommend.groupby(['posted_year']).sum().reset_index()
df_UR2API = df_UR2API[['posted_year']]
df_UR2API['usrrec_game1'] = df_UR2API.posted_year.apply(lambda x: top3_games(x,df_UserRecommend))
df_UR2API['usrrec_game2'] = df_UR2API.posted_year.apply(lambda x: top3_games(x,df_UserRecommend,2))
df_UR2API['usrrec_game3'] = df_UR2API.posted_year.apply(lambda x: top3_games(x,df_UserRecommend,3))
df_UR2API.to_csv('Data4_FuncReturn2API\\UsersRecforYear.csv',index=False)

# UsersNotRecommend

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [26]:
## Antes de crear la función, primero preparar el dataframe para mejorar el rendimiento de la función
df_UserNotRecommend = df_Gral_UserRec[~df_Gral_UserRec.recommend]
df_UserNotRecommend = df_UserNotRecommend.groupby(['posted_year', 'app_name']).sum().reset_index()
df_UserNotRecommend.drop(columns=['recommend','item_id','user_id','Neutro','Positivo'], inplace=True)
df_UserNotRecommend = df_UserNotRecommend.sort_values(by=['posted_year', 'Negativo'],ascending=[True, False])

In [27]:
## CREACIÓN DEL CSV DE CONSULTA 4 OPTIMIZADA PARA LA API
df_UNR2API = df_UserNotRecommend.groupby(['posted_year']).sum().reset_index()
df_UNR2API = df_UNR2API[['posted_year']]
df_UNR2API['usrnrec_game1'] = df_UNR2API.posted_year.apply(lambda x: top3_games(x,df_UserNotRecommend))
df_UNR2API['usrnrec_game2'] = df_UNR2API.posted_year.apply(lambda x: top3_games(x,df_UserNotRecommend,2))
df_UNR2API['usrnrec_game3'] = df_UNR2API.posted_year.apply(lambda x: top3_games(x,df_UserNotRecommend,3))
df_UNR2API.to_csv('Data4_FuncReturn2API\\UsersNotRecforYear.csv',index=False)

## Sentiment_Analysis ##

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [28]:
## CREACIÓN DE LA CATEGORIZACIÓN DEL ANÁLISIS DE SENTIMIENTO DEL USUARIO SOBRE UN JUEGO DETERMINADO
df_gamerev_SA = df_reviews.groupby(['item_id', 'sentiment_analysis']).size().unstack(fill_value=0)
df_gamerev_SA = df_gamerev_SA.reset_index(drop=False)
df_gamerev_SA.columns = ["item_id",'Negativo', 'Neutro',"Positivo"]

In [29]:
## CREACIÓN DEL CSV DE CONSULTA 5 OPTIMIZADA PARA LA API
df_Sentiment_Analysis = pd.merge(df_gamerev_SA,df_games[["item_id","release_year"]], on="item_id", how="inner")
df_SA_grouped = df_Sentiment_Analysis.groupby(['release_year']).sum().reset_index()
df_SA_API = df_SA_grouped.drop(columns=['item_id'])
df_SA_API.to_csv('Data4_FuncReturn2API\\SentAnalyforYear.csv',index=False)

# Machine Learning

Para el presente caso, se solicitó cualesquiera de los dos principales modelos de sistemas de recomendación estudiados, es decir los de item-item y el user-item. Yo escogí el segundo.

Por lo tanto, para poder entrenar a mi modelo para este sistema de recomendación se debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron. En este caso el input es un usuario y el output es una lista de juegos que se le recomienda a ese usuario, en general se explican como “A usuarios que son similares a tí también les gustó…”.

El modelo debe seguir el siguiente formato:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [30]:
# Arreglo de los dataframes para el sistema de recomendacion 
df_sentanaly = pd.read_csv('Data3_EDA_out\\SentimAnalysis.csv')
df_SAusr_group = df_sentanaly.groupby(['user_id']).sum().reset_index()
recom_maxnum= df_SAusr_group.recommend.max()
df_RecUsr_API = df_SAusr_group[df_SAusr_group.recommend==recom_maxnum]
df_RecUsr_API = df_RecUsr_API[['user_id']]

In [31]:
## CREACIÓN DE LA FUNCIÓN QUE SERVIRÁ PARA APLICAR EL MODELO DE ML
def recomendacion_usuario(user_id :str):
    # Construye una matriz de usuarios y juegos
    user_game_matrix = pd.crosstab(df_sentanaly.user_id, df_sentanaly.app_name)

    if user_id in user_game_matrix.index:
        # Encuentra el índice del usuario en la matriz
        user_index = user_game_matrix.index.get_loc(user_id)
    else:
        # user_id no está en el índice
        print("El user_id no está presente en el índice.")
        return None

    # Calcula la similitud de coseno entre los usuarios
    cosine_similarities = cosine_similarity(user_game_matrix, user_game_matrix)

    # Obtiene las similitudes de coseno para el usuario dado
    similar_users = cosine_similarities[user_index]

    # Encuentra los juegos que el usuario no ha jugado
    games_played = user_game_matrix.loc[user_id]
    unrated_games = games_played[games_played == 0].index

    # Calcula las puntuaciones de recomendación sumando las similitudes de usuarios para los juegos no jugados
    recommendation_scores = user_game_matrix.loc[:, unrated_games].multiply(similar_users, axis=0).sum(axis=0)

    # Ordena las recomendaciones por puntuación descendente
    recommendations = recommendation_scores.sort_values(ascending=False).index.tolist()

    # Obtiene la lista de los primeros 5 juegos
    top_recommendations = recommendations[:5]

    return top_recommendations

In [32]:
## CREACIÓN DEL CSV DE CONSULTA 6 OPTIMIZADA PARA LA API
df_RecUsr_API['lst_rg5'] = df_RecUsr_API.user_id.apply(lambda x: recomendacion_usuario(x))
df_RecUsr_API.to_csv('Data4_FuncReturn2API\\RecGame5forUser.csv',index=False)